In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tracker
import stats
import visualizer
import utils

import importlib

In [ ]:
#videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1.mp4"
videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1.mp4"
#videofile = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1.mp4"

# Visualize the input
cap = cv2.VideoCapture(videofile)

# Get the first frame
ret, frame = cap.read()

plt.imshow(frame)

### Feed through the tracking pipeline

In [ ]:
importlib.reload(tracker)
importlib.reload(utils)

frames = utils.loadVideo(videofile,as_gray=True)
f = tracker.determineCentroids(frames)
t = tracker.trackCentroids(f)

print(t.head())

In [ ]:
importlib.reload(tracker)

compute_segments = False
if compute_segments:
    final = tracker.segmentCells(frames, t)
else:
    final = t

print(final.head())

In [14]:
utils.saveDataFrame(final, videofile.split('.')[0] + '.csv')

### Use Visualizations

In [ ]:
importlib.reload(visualizer)

visualizer.runVisualization(videofile, final, visualization="segments")

# Run Metrics

In [11]:
#prediction = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1_predicted.csv"
#groundtruth = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1_corrected.csv"
#videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1.mp4"

prediction = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1_predicted.csv"
groundtruth = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1_corrected.csv"
videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1.mp4"

#prediction = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1_predicted.csv"
#groundtruth = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1_corrected.csv"
#videofile = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1.mp4"

# Load as Pandas DataFrame
pred_src = utils.loadDataFrame(prediction)
gt_src = utils.loadDataFrame(groundtruth)


In [42]:
# Adhoc adjustment for testing
#pred_src = gt_src

### Preprocessing

In [12]:
import pandas as pd

def dropDuplicates(df):
      
    # Detect duplicate rows based on 'col1' and 'col2'
    #duplicate_rows = df[df.duplicated(subset=['frame', 'sperm'], keep='first')]
    result = df.drop_duplicates(subset=['frame', 'sperm'], keep='first')

    return result

def interpolateTracks(df):

    result = df.copy()

    for sperm in range(0, df['sperm'].max() + 1):
        sperm_frames = df[df['sperm'] == sperm]['frame'].values
        if len(sperm_frames) > 1:
            birth = np.amin(sperm_frames)
            death = np.amax(sperm_frames)


            # Find if the sperm exists for all frames
            if len(sperm_frames) != death - birth + 1:
                #print("Missing frames for sperm: ", sperm)
                #print("Birth: ", birth, ", Death: ", death)
                #print("Frames: ", sperm_frames)

                for j in range(birth, death + 1):
                    if j not in sperm_frames:
                 
                        # Find closest frame after the missing frame
                        before = np.amax(sperm_frames[np.where(sperm_frames < j)])
                        after = np.amin(sperm_frames[np.where(sperm_frames > j)])

                        # interpolate x and y
                        before_x = df[(df['sperm'] == sperm) & (df['frame'] == before)]['x'].values[0]
                        before_y = df[(df['sperm'] == sperm) & (df['frame'] == before)]['y'].values[0]
                        after_x = df[(df['sperm'] == sperm) & (df['frame'] == after)]['x'].values[0]
                        after_y = df[(df['sperm'] == sperm) & (df['frame'] == after)]['y'].values[0]

                        x = before_x + (after_x - before_x) * (j - before) / (after - before)
                        y = before_y + (after_y - before_y) * (j - before) / (after - before)

                        #print("Adding frame: ", j)
                        #gt = gt.append({'frame': j, 'sperm': sperm, 'x': x, 'y': y}, ignore_index=True)
                        result = pd.concat([result, pd.DataFrame([[j, sperm, x, y]], columns=['frame', 'sperm', 'x', 'y'])], ignore_index=True)

    return result

def filterSperm(df, epsilon=5.0):

    filter_list = []

    # Filter out sperm that are not moving
    for sperm in df['sperm'].unique():
        all_locs = df[df['sperm'] == sperm]
        # Determine the mean sperm location
        mean_loc = all_locs[['x', 'y']].mean()

        # If the furthest away is less than e, remove the sperm
        all_dists = all_locs[['x', 'y']].sub(mean_loc)
        all_dists = all_dists.pow(2)
        all_dists = all_dists.sum(axis=1)
        all_dists = all_dists.pow(0.5)
        if all_dists.max() < epsilon:
            filter_list.append(sperm)

    # Filter out sperm that are not moving
    df = df[~df['sperm'].isin(filter_list)]

    return df

gt_u = dropDuplicates(gt_src)
pred_u = dropDuplicates(pred_src)

print(len(gt_src), len(gt_u))
print(len(pred_src), len(pred_u))

gt = interpolateTracks(gt_u)
pred = interpolateTracks(pred_u)

print(len(gt_src))
print(len(gt))

pred_filter = filterSperm(pred)
gt_filter = filterSperm(gt)

print(len(pred['sperm'].unique()))
print(len(pred_filter['sperm'].unique()))

print(len(gt['sperm'].unique()))
print(len(gt_filter['sperm'].unique()))

6909 6904
8157 8157
6909
6992
51
44
43
36


In [44]:
# Visualize
visualizer.runVisualization(videofile, pred_src, visualization="flow")

(118, 3)
Video Finished.


In [4]:
visualizer.runVisualization(videofile, pred, visualization="flow")

(118, 3)
Video Finished.


In [51]:
visualizer.runVisualization(videofile, pred_filter, visualization="flow")

(100, 3)
Video Finished.


In [ ]:
visualizer.runVisualization(videofile, gt_src, visualization="flow")

(94, 3)
Video Finished.


In [ ]:
visualizer.runVisualization(videofile, gt, visualization="flow")

(94, 3)
Video Finished.


In [50]:
visualizer.runVisualization(videofile, gt_filter, visualization="flow")

(109, 3)
Video Finished.


### Unlabeled Metrics

In [ ]:
import pandas as pd

def spermPerFrame(df, return_summary=False):
    # Count occurrences of each sperm count in each frame
    frame_counts = {}
    max_sperm = df["sperm"].max()
    sperm_labels = list(range(max_sperm + 1))  # Generate labels from 0 to max_sperm

    for frame in df["frame"].unique():
        frame_counts[frame] = df[df["frame"] == frame]["sperm"].value_counts().reindex(sperm_labels).fillna(0)

    # Create a DataFrame for sperm counts per frame
    frame_summary = pd.DataFrame.from_dict(frame_counts, orient='index', columns=sperm_labels)
    frame_summary.index.name = "frame"
    frame_summary.reset_index(inplace=True)

    # Add a column for total sperms observed in each frame
    frame_summary["TotalSperms"] = frame_summary[sperm_labels].sum(axis=1)

    # Extract ["TotalSperms"] as numpy array
    array = frame_summary["TotalSperms"].values

    if return_summary:
        return array, frame_summary
    else:
        return array

def framesPerSperm(df, return_summary=False):
    sperm_counts = {}
    max_frame = df["frame"].max()
    frame_labels = list(range(max_frame + 1)) # Generate labels from 0 to max_frame

    for sperm in df["sperm"].unique():
        sperm_counts[sperm] = df[df["sperm"] == sperm]["frame"].value_counts().reindex(frame_labels).fillna(0)

    # Create a DataFrame for frame counts per sperm
    sperm_summary = pd.DataFrame.from_dict(sperm_counts, orient='index', columns=frame_labels)
    sperm_summary.index.name = "sperm"
    sperm_summary.reset_index(inplace=True)

    # Add a column for total sperms observed in each frame
    sperm_summary["TotalFrames"] = sperm_summary[frame_labels].sum(axis=1)

    # Remove sperms that are not observed in any frame
    sperm_summary = sperm_summary[sperm_summary["TotalFrames"] > 0]

    # Extract ["TotalFrames"] as numpy array
    array = sperm_summary["TotalFrames"].values

    if return_summary:
        return array, sperm_summary
    else:
        return array


pred_spf = spermPerFrame(pred)
gt_spf = spermPerFrame(gt)

#print(pred_frames)
#print(gt_frames)

pred_fps = framesPerSperm(pred)
gt_fps = framesPerSperm(gt)

#print(pred_sperm)
#print(gt_sperm)

max_sperm = int(max(max(pred_spf), max(gt_spf)))

plt.hist(pred_spf, bins=range(0, max_sperm, 1), alpha=0.5, color='red', label='Predicted')
plt.hist(gt_spf, bins=range(0, max_sperm, 1), alpha=0.5, color='blue',label='Ground Truth')
plt.show()

max_frames = int(max(max(pred_fps), max(gt_fps)))

plt.hist(pred_fps, bins=range(0, max_frames, 10), alpha=0.5, color='red', label='Predicted')
plt.hist(gt_fps, bins=range(0, max_frames, 10), alpha=0.5, color='blue',label='Ground Truth')
plt.show()

print("Sperm Per Frame Pred - Mean: ", np.mean(pred_spf), ", Std: ", np.std(pred_spf), ", Normalized Mean: ", np.mean(pred_spf)/max_sperm, ", Normalized Std: ", np.std(pred_spf)/max_sperm)
print("Sperm Per Frame GT - Mean: ", np.mean(gt_spf), ", Std: ", np.std(gt_spf), ", Normalized Mean: ", np.mean(gt_spf)/max_sperm, ", Normalized Std: ", np.std(gt_spf)/max_sperm)
print("Frames Per Sperm Pred - Mean: ", np.mean(pred_fps), ", Std: ", np.std(pred_fps), ", Normalized Mean: ", np.mean(pred_fps)/max_frames, ", Normalized Std: ", np.std(pred_fps)/max_frames)
print("Frames Per Sperm GT - Mean: ", np.mean(gt_fps), ", Std: ", np.std(gt_fps), ", Normalized Mean: ", np.mean(gt_fps)/max_frames, ", Normalized Std: ", np.std(gt_fps)/max_frames)



### Labeled Metrics

In [13]:
# Create track data needed for calculation (Label, Birth Frame, Death Frame, 0 (no parent))

def makeTrackData(df):
    tracks = []

    for s in range(0, df['sperm'].max() + 1):
        cur_frames = df[df['sperm'] == s]['frame'].values
        if len(cur_frames) > 0:
            cur_birth = np.amin(cur_frames)
            cur_death = np.amax(cur_frames)
            cur_parent = 0
            cur_track = [s, cur_birth, cur_death, cur_parent]
            tracks.append(cur_track)

    return np.array(tracks)

pred_tracks = makeTrackData(pred)
gt_tracks = makeTrackData(gt)
pred_filter_tracks = makeTrackData(pred_filter)
gt_filter_tracks = makeTrackData(gt_filter)

#ref_tracks = []

#for s in range(0, gt['sperm'].max() + 1):
#    cur_frames = gt[gt['sperm'] == s]['frame'].values
#    if len(cur_frames) > 0:
#        cur_birth = np.amin(cur_frames)
#        cur_death = np.amax(cur_frames)
#        cur_parent = 0
#        cur_track = [s, cur_birth, cur_death, cur_parent]
#        ref_tracks.append(cur_track)

#ref_tracks = np.array(ref_tracks)

#comp_tracks = []

#for s in range(0, pred['sperm'].max() + 1):
#    cur_frames = pred[pred['sperm'] == s]['frame'].values
#    if len(cur_frames) > 0:
#        cur_birth = np.amin(cur_frames)
#        cur_death = np.amax(cur_frames)
#        cur_parent = 0
#        cur_track = [s, cur_birth, cur_death, cur_parent]
#        comp_tracks.append(cur_track)

#comp_tracks = np.array(comp_tracks)

In [14]:
from  scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist


def makeTrajectoryData(pred,gt):

    # Use Hungarian Algorithm to find best track matches between pred and gt in each frame
    labels_ref = []
    labels_comp = []
    mapped_ref = []
    mapped_comp = []

    # For each frame
    for f in range(0, pred['frame'].max() + 1):

        mapped_ref_frame = []
        mapped_comp_frame = []

        # Get the labels in the frame
        ref_data = gt[gt['frame'] == f][['sperm','x','y']].values
        comp_data = pred[pred['frame'] == f][['sperm','x','y']].values

        labels_ref_frame = ref_data[:,0].astype(int)
        labels_comp_frame = comp_data[:,0].astype(int)

        ref_centroids = ref_data[:,1:]
        comp_centroids = comp_data[:,1:]

        #labels_ref_frame = gt[gt['frame'] == f]['sperm'].values
        #labels_comp_frame = pred[pred['frame'] == f]['sperm'].values
        #print(labels_ref_frame.dtype, labels_ref_frame2.dtype)

        # Get the centroids in the frame
        #ref_centroids = gt[gt['frame'] == f][['x', 'y']].values
        #comp_centroids = pred[pred['frame'] == f][['x', 'y']].values

        # Compute the distance matrix
        dist_matrix = cdist(ref_centroids, comp_centroids)

        #print(dist_matrix.shape)

        #import matplotlib.pyplot as plt
        #plt.imshow(dist_matrix);plt.show()
        #print(np.amin(dist_matrix))

        # Use Hungarian Algorithm to find best matches
        row_ind, col_ind = linear_sum_assignment(dist_matrix)

        if (len(row_ind) != len(np.unique(row_ind))):
            print("row issue!")
            print(row_ind.shape,np.unique(row_ind).shape)

        if (len(col_ind) != len(np.unique(col_ind))):
            print("col issue!")
            print(col_ind.shape,np.unique(col_ind).shape)

        if len(labels_ref_frame) != len(np.unique(labels_ref_frame)):
            print("labels_ref issue!")
        
        if len(labels_comp_frame) != len(np.unique(labels_comp_frame)):
            print("labels_comp issue!")

        #for r, c in zip(row_ind, col_ind):
        #    print(r,c)

        # Save the matches
        for r, c in zip(row_ind, col_ind):
            mapped_ref_frame.append(labels_ref_frame[r])
            mapped_comp_frame.append(labels_comp_frame[c])

            #labels_ref_frame.append(ref_labels[r])
            #labels_comp_frame.append(comp_labels[c])
            #mapped_ref_frame.append(gt[(gt['frame'] == f) & (gt['sperm'] == ref_labels[r])]['sperm'].values[0])
            #mapped_comp_frame.append(pred[(pred['frame'] == f) & (pred['sperm'] == comp_labels[c])]['sperm'].values[0])

        labels_ref.append(labels_ref_frame)
        labels_comp.append(labels_comp_frame)
        mapped_ref.append(mapped_ref_frame)
        mapped_comp.append(mapped_comp_frame)

    traj = {}
    traj['labels_ref'] = labels_ref
    traj['labels_comp'] = labels_comp
    traj['mapped_ref'] = mapped_ref
    traj['mapped_comp'] = mapped_comp

    #print(labels_ref)
    #print(labels_comp)
    #print(mapped_ref)
    #print(mapped_comp)

    return traj

traj = makeTrajectoryData(pred,gt)
traj_filter = makeTrajectoryData(pred_filter,gt_filter)

In [15]:
from ctc_metrics.utils.representations import merge_tracks, count_acyclic_graph_correction_operations

def appendMergedTrajectory(ref_tracks, comp_tracks, traj):

    new_tracks, new_labels, new_mapped = merge_tracks(
        ref_tracks, traj["labels_ref"], traj["mapped_ref"])
    traj["ref_tracks_merged"] = new_tracks
    traj["labels_ref_merged"] = new_labels
    traj["mapped_ref_merged"] = new_mapped
    new_tracks, new_labels, new_mapped = merge_tracks(
        comp_tracks, traj["labels_comp"], traj["mapped_comp"])
    traj["comp_tracks_merged"] = new_tracks
    traj["labels_comp_merged"] = new_labels
    traj["mapped_comp_merged"] = new_mapped

    return traj

traj = appendMergedTrajectory(gt_tracks, pred_tracks, traj)
traj_filter = appendMergedTrajectory(gt_filter_tracks, pred_filter_tracks, traj_filter)


In [16]:
from ctc_metrics.utils.representations import count_acyclic_graph_correction_operations
from ctc_metrics.metrics import (
    valid, det, seg, tra, ct, tf, bc, cca, mota, hota, idf1, chota, mtml, faf,
    op_ctb, op_csb, bio, op_clb, lnk
)

def computeMetrics(ref_tracks, comp_tracks, traj):

    graph_operations = \
                count_acyclic_graph_correction_operations(
                    ref_tracks, comp_tracks,
                    traj["labels_ref"], traj["labels_comp"],
                    traj["mapped_ref"], traj["mapped_comp"]
                )

    #print(graph_operations)

    results = {}
    results["DET"] = det(**graph_operations)
    _tra, _aogm, _aogm0 = tra(**graph_operations)
    results["TRA"] = _tra
    results["AOGM"] = _aogm
    results["AOGM_0"] = _aogm0
    for key in ("NS", "FN", "FP", "ED", "EA", "EC"):
        results[f"AOGM_{key}"] = graph_operations[key]

    results["LNK"] = lnk(**graph_operations)

    results["CT"] = ct(
                comp_tracks, ref_tracks,
                traj["labels_ref"], traj["mapped_ref"], traj["mapped_comp"])

    results["TF"] = tf(
        ref_tracks,
        traj["labels_ref"], traj["mapped_ref"], traj["mapped_comp"])


    results.update(mota(
        traj["labels_ref_merged"], traj["labels_comp_merged"],
        traj["mapped_ref_merged"], traj["mapped_comp_merged"]))

    results.update(hota(
        traj["labels_ref_merged"], traj["labels_comp_merged"],
        traj["mapped_ref_merged"], traj["mapped_comp_merged"]))

    results.update(idf1(
        traj["labels_ref_merged"], traj["labels_comp_merged"],
        traj["mapped_ref_merged"], traj["mapped_comp_merged"]))
    
    return results

results = computeMetrics(gt_tracks, pred_tracks, traj)
results_filter = computeMetrics(gt_filter_tracks, pred_filter_tracks, traj_filter)


In [17]:
for key,val in results.items():
    print(key, val)

DET 0.9829948512585812
TRA 0.8076571222314188
AOGM 15453.5
AOGM_0 80343.5
AOGM_NS 0
AOGM_FN 0
AOGM_FP 1189
AOGM_ED 5299
AOGM_EA 5977
AOGM_EC 0
LNK 0.0
CT 0.46808510638297873
TF 0.9229395506934749
MOTA 0.8260869565217391
TP 6992
FP 1189
FN 0
IDSW 27
MULTI-ASSIGNMENTS 0
Precision 0.854663244102188
Recall 1.0
HOTA 0.8793668644077607
IDF1 0.866802873525341
IDP 0.8038137147048038
IDR 0.9405034324942791
IDTP 6576.0
IDFP 1605.0
IDFN 416.0


In [18]:
for key,val in results_filter.items():
    print(key, val)

DET 0.9798942731277533
TRA 0.8067889922326077
AOGM 12599.0
AOGM_0 65208.5
AOGM_NS 0
AOGM_FN 0
AOGM_FP 1141
AOGM_ED 4189
AOGM_EA 4846
AOGM_EC 0
LNK 0.0
CT 0.5
TF 0.9202412999566872
MOTA 0.7948898678414097
TP 5675
FP 1141
FN 0
IDSW 23
MULTI-ASSIGNMENTS 0
Precision 0.832599765258216
Recall 1.0
HOTA 0.8619705068542
IDF1 0.8463693859578897
IDP 0.7755281690140845
IDR 0.931453744493392
IDTP 5286.0
IDFP 1530.0
IDFN 389.0
